# Cleaning Temperature Data
### Karl Madl
### 2021 September 01

---
## Imports

We'll import the *Pandas* and *mysq.connector* packages to import database data into a Pandas dataframe which we'll name **raw_temperature_df** (raw temperature dataframe). We'll also import a dictionary containing the login information to the MySQL server as **CREDS**.

Note that we'll leave **id** out of our SQL *SELECT* statement since Pandas provides automatic row indexing. These new indexes will be equal to the orignal **id** of the data minus 1.

In [ ]:
import pandas as pd
import mysql.connector as connector
from database_credentials import MySQL_credentials as CREDS

connection = connector.connect(
    host = CREDS['host'],
    user = CREDS['user'],
    password = CREDS['password'],
    database = CREDS['database']
)

raw_temperature_df = pd.read_sql(f"SELECT inside_temperature, outside_temperature, time, date FROM {CREDS['table']}", con=connection)

---

## Data Types

Next, we'll check the shape of the dataframe to confirm we imported all of the rows. We should have, at the time of writing, 347 rows. This is confirmed by accessing the *shape* attribute.

In [ ]:
raw_temperature_df.shape

We'll check that our columns (attributes) are of the proper data type by accessing the *dtypes* attribute of the dataframe. In this case, the **date** column was incorrectly typed as an *object* (string) since Pandas doesn't support the *date* type that the **date** column was stored as in the database.

Note that Pandas also doesn't support the *time* type that the **time** column is stored as so it has been converted to a *timedelta*. This is fine for the purposes of our analysis.

In [ ]:
raw_temperature_df.dtypes

To convert the **date** column to a *datetime* we'll use the *to_datetime* function that Pandas offers. 

Another route to take would be re-querying the data, *CAST*ing the **date** column as *datetime* but that would be less computation efficient and less time efficient. This would also likely muddle the clarity of the data cleaning process.

Secondly, we'll create extra columns containing the year, month, and day of the month for each observation.

In [ ]:
raw_temperature_df['date'] = pd.to_datetime(raw_temperature_df['date'])  # convert date column type from object to datetime

raw_temperature_df['year'] = raw_temperature_df['date'].map(lambda x: x.year)
raw_temperature_df['month'] = raw_temperature_df['date'].map(lambda x: x.month)
raw_temperature_df['day'] = raw_temperature_df['date'].map(lambda x: x.day)

A final check of the data types in each column reveals exactly the desired outcome. The **year**, **month**, and **day** columns are as integers but this is fine for our purposes.

In [ ]:
raw_temperature_df.dtypes

While data is only uploaded to the database if there are no null values in the observation, we should remove rows in the dataframe containing NaN values, the Pandas equivalent for null values, to be confident we won't raise any arithmetic exceptions during the analysis phase. 

In [ ]:
raw_temperature_df.dropna(axis='index')

---
## Consistency of Data and Duplicates

We should make sure that the amount of data in each group that we'll be analyzing is consistent so as not to introduce bias into our analysis.
This isn't a necessary prerequisite if we wish to perform an ANOVA test, so long as the variance between the groups is similar. [^1] For this reason we'll also check the variance of the groups by looking at the standard deviation for each. Let's first start with the **time** groups.

In [ ]:
raw_temperature_df.groupby(['time']).describe().loc[:, ['inside_temperature', 'outside_temperature']]

The first thing to notice is that, there are some observations that were recorded at odd times. This could be caused due to a power outage delaying the running of the data collection script or an error in uploading to the database. We can allow for a margin of error of a minute, since temperature changes are typically negligible on the timescales of seconds. Upon further examination, the discrepancy between the count of observations at midnight (00:00:00) and the other times seems a bit conspicuous, especially if we were to assign the times between 00:00:04 and 00:00:20 to also be midnight observations.

A more meticulous manual look at the data shows an interesting pattern between indices 64 and 80 and it becomes clear what occurred. From the latter half of 2021-07-06 through 2021-07-08, all times were formatted incorrectly and thus we can adjust them using the *replace* Pandas function. We'll then use the same function to adjust the 04:01:00, 08:01:00, and 20:01:00 timed observations. We'll also create a new dataframe **fixed_times_df** to move forward.

Finally, we'll use the *drop* function to eliminate the observation recorded at 13:19:00 and use *drop_duplicates* based on **date** and **time** to eliminate any conflicting observations and repeated entries.

In [ ]:
fixed_times_df = raw_temperature_df.replace(
    to_replace=pd.to_timedelta(['00:00:04', '00:00:08', '00:00:12', '00:00:16', '00:00:20']), 
    value=pd.to_timedelta(['04:00:00', '08:00:00', '12:00:00', '16:00:00', '20:00:00']),
    )

fixed_times_df.replace(
    to_replace=pd.to_timedelta(['04:01:00', '08:01:00', '20:01:00']), 
    value=pd.to_timedelta(['04:00:00', '08:00:00', '20:00:00']),
    inplace=True
    )

fixed_times_df.drop(fixed_times_df[fixed_times_df['time'] == pd.to_timedelta('13:19:00')].index, inplace=True)

fixed_times_df.drop_duplicates(['date', 'time'], keep='first', inplace=True)

Checking again on the description of the dataframe, grouped by time, we now have more consistent group sizes and standard deviations (a measure of variance) across groups.

In [ ]:
fixed_times_df.groupby(['time']).describe().loc[:, ['inside_temperature', 'outside_temperature']]

We'll check the same, now grouping by **day**, **month**, and **year** (which wil have the same output as **season** since data collection has only occurred during summer). All of these have consistent standard deviations and ranges. There is some inconsistency between **day** groups and **month** groups but, because the variance in the temperature readings is similar, we can proceed. It should be note, however, that any statistical tests run will only have the power based on the smallest group in the group pool.

In [ ]:
fixed_times_df.groupby(['day']).describe().loc[:, ['inside_temperature', 'outside_temperature']]

In [ ]:
fixed_times_df.groupby(['month']).describe().loc[:, ['inside_temperature', 'outside_temperature']]

In [ ]:
fixed_times_df.groupby(['year']).describe().loc[:, ['inside_temperature', 'outside_temperature']]

---
## Conclusion

The data is now sufficiently cleaned and may be used for analysis.

In [ ]:
clean_df = fixed_times_df